In [39]:
import numpy as np

# Define the objective function to minimize (e.g., total waiting time, number of stops)
def objective_function(signal_timings):
    # Simulate traffic flow and calculate the objective value based on signal timings
    # Your simulation logic goes here
    objective_value = simulate_traffic(signal_timings)
    return objective_value

def simulate_traffic(signal_timings):
    # Ensure signal_timings is a list
    if not isinstance(signal_timings, list):
        raise ValueError("signal_timings must be a list of signal timings")
    
    # Ensure all elements of signal_timings are lists with at least two elements
    for phase_timing in signal_timings:
        if not isinstance(phase_timing, list) or len(phase_timing) < 2:
            raise ValueError("Each signal phase timing must be a list with at least two elements: green time and yellow time")
    
    # Example: Simulate traffic flow and calculate the objective value based on signal timings
    # For demonstration purposes, let's assume a simple metric such as total waiting time
    total_waiting_time = 0
    
    # Iterate over each signal phase
    for phase_timing in signal_timings:
        # Example: Calculate waiting time for each phase based on green time, yellow time, etc.
        # You should replace this with a more realistic calculation based on traffic flow models
        green_time = phase_timing[0]  # Green time for the current phase
        yellow_time = phase_timing[1]  # Yellow time for the current phase
        
        # Example: Calculate waiting time based on traffic flow model (e.g., traffic queues, vehicle arrivals)
        # For demonstration purposes, let's assume a random waiting time between 0 and 100 seconds per phase
        waiting_time = np.random.randint(0, 100)
        total_waiting_time += waiting_time
    
    # Calculate the overall objective value based on the total waiting time
    objective_value = total_waiting_time
    return objective_value

# Particle Swarm Optimization algorithm
def particle_swarm_optimization(objective_function, num_dimensions, num_particles, max_iter):
    # Initialize particles' positions and velocities
    particles_position = np.random.rand(num_particles, num_dimensions)  # Random initialization
    particles_velocity = np.random.rand(num_particles, num_dimensions)  # Random initialization

    # Initialize personal best positions and values for each particle
    particles_best_position = particles_position.copy()
    particles_best_value = np.zeros(num_particles)
    for i in range(num_particles):
        particles_position_list = [list(particles_position[i])]  # Convert to list of lists
        particles_best_value[i] = objective_function(particles_position_list)

    # Initialize global best position and value using the first particle's values
    global_best_value = particles_best_value[0]
    global_best_position = particles_best_position[0]

    # PSO parameters
    inertia_weight = 0.5
    cognitive_weight = 1.5
    social_weight = 1.5

    # Iterate through optimization process
    for _ in range(max_iter):
        # Update particles' velocities and positions
        for i in range(num_particles):
            # Update velocity
            r1 = np.random.rand(num_dimensions)
            r2 = np.random.rand(num_dimensions)
            cognitive_component = cognitive_weight * r1 * (particles_best_position[i] - particles_position[i])
            social_component = social_weight * r2 * (global_best_position - particles_position[i])
            particles_velocity[i] = inertia_weight * particles_velocity[i] + cognitive_component + social_component

            # Update position
            particles_position[i] += particles_velocity[i]

            # Ensure particle positions are within bounds (e.g., non-negative)
            particles_position[i] = np.clip(particles_position[i], 0, None)

            # Update personal best position and value for particle i
            current_value = objective_function([list(particles_position[i])])  # Convert to list of lists
            if current_value < particles_best_value[i]:
                particles_best_position[i] = particles_position[i]
                particles_best_value[i] = current_value

                # Update global best position and value if needed
                if current_value < global_best_value:
                    global_best_value = current_value
                    global_best_position = particles_position[i]

    return global_best_position, global_best_value

# Example usage
if __name__ == "__main__":
    # Example parameters
    num_dimensions = 4  # Number of dimensions (e.g., signal timings for different phases)
    num_particles = 50  # Number of particles in the swarm
    max_iter = 100  # Maximum number of iterations

    # Run PSO optimization
    best_solution, best_value = particle_swarm_optimization(objective_function, num_dimensions, num_particles, max_iter)

    print("Best solution (signal timings):", best_solution)
    print("Best objective value:", best_value)


Best solution (signal timings): [0.95070189 0.98214452 0.58885084 1.51841428]
Best objective value: 0
